In [1]:
import requests as r
from flask import request 
from bs4 import BeautifulSoup as bs
# 用于伪造headers，访问者
from faker import Faker
import random

In [2]:
agent = Faker('zh-CN').user_agent()
def get_cityCode(province='',city=''):
    city_json = 'https://www.zhipin.com/wapi/zpCommon/data/city.json'
    result = r.get(city_json,
                headers={'User-Agent':agent}).json()
    if (result['message'] == 'Success'):
        cityList = result['zpData']['cityList']
        for p in cityList:
            if p['name'] == province:
                sub = p['subLevelModelList']
                flag = 1
                for c in sub:
                    if city == c['name']:
                        return p['code'],c['code']  
                return p['code']
        else:
            print('找不到对象')
            return None
    
    else:
        print('调用boss直聘的获取地区编码的api有问题！',city)

In [47]:
get_cityCode('河北','保定')

(101090000, 101090200)

In [56]:
city = get_cityCode('广东','广州')[1]

# 数据上限399条
def get_job(job,city):
    proxies = [{'http':'http://47.107.175.190:8000'},
          {'http':'http://163.204.242.64:9999'},
          {'http':'http://47.112.214.45:8000'},
        {'http':'http://163.204.242.181:9999'},
        {'http':'http://163.204.241.214:9999'}]
    
    job_json = 'https://www.zhipin.com/mobile/jobs.json'
    params = {
        'query':job,
        'page':1,
        'city':city
    }
    job_data = []
    while 1:
        pro = random.choice(proxies)
        response = r.get(job_json,
              params=params,
              headers={'User-Agent':agent},
            proxies=pro).json()
        
        try:
            
            if response['html']:
                params['page'] +=1
                html = bs(response['html'],'html.parser').find_all(class_='item')
                for h in html:
                # item字典一定要写在循环里面且每遍历完一个职位条目就重新清空字典、保存在job列表当中
                # 否则会重复将最后的值，反复追加到job列表，可见变量名只是个字典的映射。
                    item = {}
                    item['salary'] = h.find('span',"salary").string 
                    item['company'] = h.find('div','name').string
                    item['jobName'] = h.find('div','title').find('h4').string
                    msg = h.find('div','msg').find_all('em')
                    for em in range(3):
                        item['city'] = msg[0].string
                        item['experience'] = msg[1].string
                        item['education'] = msg[2].string
                    job_data.append(item)
            elif response['resmsg'] != '成功':
                print('调用boss直聘获取工作列表api有问题！或者是网络、服务器有问题')
                continue
            elif response['html'] == '' and params['page']==1:
                print('用户输入的关键词有误，找不到对象！')
                return None
                break
            else:
                break
        # 返回的json数据结果没有html键，说明ip已被封
        except KeyError:
            print(response)
            break
        # 连接代理服务出现异常时，就把那个代理ip从列表中删除并继续用另一个代理ip
        except (ConnectTimeout,ProxyError,ReadTimeout):
            proxies.remove(pro)
            continue
    return job_data

demo_job= get_job('销售经理',get_cityCode('广东','梅州')[1])

In [57]:
print(demo_job)

[{'salary': '10-15K', 'company': '百纳行集团', 'jobName': '销售经理', 'city': '梅州', 'experience': '经验不限', 'education': '本科'}, {'salary': '8-13K', 'company': '云充吧', 'jobName': '销售经理', 'city': '梅州', 'experience': '经验不限', 'education': '高中'}, {'salary': '10-11K', 'company': '梅州百纳行', 'jobName': '销售经理', 'city': '梅州', 'experience': '经验不限', 'education': '学历不限'}, {'salary': '6-7K', 'company': '靓家居', 'jobName': '储备销售经理', 'city': '梅州', 'experience': '3-5年', 'education': '大专'}, {'salary': '5-10K', 'company': '梅州广益', 'jobName': '销售经理', 'city': '梅州', 'experience': '1年以内', 'education': '学历不限'}, {'salary': '5-8K', 'company': '长兴润德教育', 'jobName': '销售经理', 'city': '梅州', 'experience': '1-3年', 'education': '大专'}, {'salary': '5-6K', 'company': '智宏信息', 'jobName': '销售经理', 'city': '梅州', 'experience': '1-3年', 'education': '大专'}, {'salary': '8-15K', 'company': '靓家居', 'jobName': '销售经理', 'city': '梅州', 'experience': '3-5年', 'education': '大专'}, {'salary': '15-30K', 'company': '科大讯飞华南', 'jobName': '城市销售经理（梅州）', 'city': '梅州', 

In [6]:
def count_num(data,key='education'):
    edu = {}
    length = len(data)
    for j in range(length):
        k=data[j][key]
        edu[k] = edu.get(k,0)+1
    return sorted(edu.items(),
                  key=lambda x : x[1],
                  reverse=True)

In [15]:
html = bs(response['html'],'html.parser').find_all(class_ = 'item')
for h in html:
    print(h.find('div','title').find('h4').string)
    

架构师（区块链）
区块链工程师
区块链开发
金融安全密码学（区块链支付）
区块链高级算法工程师
区块链工程师
区块链工程师
PHP主管（区块链）
产品设计师<区块链应用>
区块链开发——高级
产品经理（区块链社区方向）
区块链项目售前咨询
区块链-开发
㊣㊣区块链开发CTO
安全密码学（区块链支付）


In [8]:
proxies = [{'http':'http://47.107.175.190:8000'},
      {'http':'http://163.204.242.64:9999'},
      {'http':'http://47.112.214.45:8000'},
    {'http':'http://163.204.242.181:9999'},
    {'http':'http://163.204.241.214:9999'},
    {'http':'http://58.62.44.103:8088'}]

job_json = 'https://www.zhipin.com/mobile/jobs.json'

params = {
    'query':'区块链',
    'page':1,
    'city':city
}
pro = random.choice(proxies)
response = r.get(job_json,
      params=params,
      headers={'User-Agent':agent},
    proxies=pro).json()

print(response)

{'hasMore': True, 'resmsg': '成功', 'rescode': 1, 'html': '<li class="item">\n            <a href="/job_detail/9faf5ed7f62e5cd91HZ529i8ElM~.html" ka="job_list_0" data-itemId="0" data-lid="1CVhufHDagb.search.1" data-index="0">\n                    <img src="https://img.bosszhipin.com/beijin/upload/com/logo/20190723/b2e540c134031268a87d32020a0db4a7199e687ea16fba23b90d928846187393.png?x-oss-process=image/resize,w_120,limit_0"/>\n                <div class="text">\n                    <div class="title"><h4>架构师（区块链）</h4><span class="salary">15-30K</span></div>\n                    <div class="name">小马</div>\n                    <div class="msg"><em>广州</em><em>经验不限</em><em>学历不限</em></div>\n                </div>\n            </a>\n        </li>\n        <li class="item">\n            <a href="/job_detail/e6805944b35704ef0nN42Nq5Flo~.html" ka="job_list_0" data-itemId="0" data-lid="1CVhufHDagb.search.2" data-index="1">\n                    <img src="https://img.bosszhipin.com/beijin/mcs/chatpho

In [46]:
city_json = 'https://www.zhipin.com/wapi/zpCommon/data/city.json'
adv = r.get(city_json,
                headers={'User-Agent':agent}).json()['zpData']['cityList']

# 创建一个市级行政区域与省级行政区的映射，方便后面统计各省的职位数量
pt = {}
for ad in adv:
    for a in ad['subLevelModelList']:
        pt[a['name']] = ad['name']
print(pt)

{'北京': '北京', '上海': '上海', '天津': '天津', '重庆': '重庆', '哈尔滨': '黑龙江', '齐齐哈尔': '黑龙江', '牡丹江': '黑龙江', '佳木斯': '黑龙江', '绥化': '黑龙江', '黑河': '黑龙江', '伊春': '黑龙江', '大庆': '黑龙江', '七台河': '黑龙江', '鸡西': '黑龙江', '鹤岗': '黑龙江', '双鸭山': '黑龙江', '大兴安岭': '黑龙江', '长春': '吉林', '吉林': '吉林', '四平': '吉林', '通化': '吉林', '白城': '吉林', '辽源': '吉林', '松原': '吉林', '白山': '吉林', '延边': '吉林', '沈阳': '辽宁', '大连': '辽宁', '鞍山': '辽宁', '抚顺': '辽宁', '本溪': '辽宁', '丹东': '辽宁', '锦州': '辽宁', '营口': '辽宁', '阜新': '辽宁', '辽阳': '辽宁', '铁岭': '辽宁', '朝阳': '辽宁', '盘锦': '辽宁', '葫芦岛': '辽宁', '呼和浩特': '内蒙古', '包头': '内蒙古', '乌海': '内蒙古', '通辽': '内蒙古', '赤峰': '内蒙古', '鄂尔多斯': '内蒙古', '呼伦贝尔': '内蒙古', '巴彦淖尔': '内蒙古', '乌兰察布': '内蒙古', '锡林郭勒': '内蒙古', '兴安盟': '内蒙古', '阿拉善': '内蒙古', '石家庄': '河北', '保定': '河北', '张家口': '河北', '承德': '河北', '唐山': '河北', '廊坊': '河北', '沧州': '河北', '衡水': '河北', '邢台': '河北', '邯郸': '河北', '秦皇岛': '河北', '太原': '山西', '大同': '山西', '阳泉': '山西', '晋中': '山西', '长治': '山西', '晋城': '山西', '临汾': '山西', '运城': '山西', '朔州': '山西', '忻州': '山西', '吕梁': '山西', '西安': '陕西', '咸阳': '陕西', '延安': '陕西', '榆林': '陕西', '渭南': '陕西',

In [17]:
job = get_job('区块链',get_cityCode('广东','佛山'))

In [37]:
job_data = {'salary':[],'jobName':[],'education':[],'experience':[]}
for j in job:
    job_data['salary'].append(j['salary'])
    job_data['jobName'].append(j['jobName'])
    job_data['education'].append(j['education'])
    job_data['experience'].append(j['experience'])
print(job_data)

{'salary': ['22-35K', '20-40K', '15-30K', '13-25K', '40-70K', '25-50K', '30-60K', '20-40K', '10-15K', '25-50K', '12-24K', '20-35K', '20-40K', '20-40K', '20-40K', '15-30K', '20-30K', '35-65K', '50-80K', '30-60K', '25-50K', '13-25K', '6-10K', '12-20K', '18-28K', '20-40K', '20-35K', '12-20K', '20-35K', '50-80K', '15-30K', '15-30K', '12-20K', '25-50K', '15-25K', '20-30K', '15-30K', '17-30K', '160-210/天', '10-15K', '30-60K', '20-35K', '15-30K', '30-60K', '15-25K', '15-25K', '35-50K', '15-30K', '12-24K', '6-10K', '6-10K', '15-30K', '25-50K', '15-30K', '25-50K', '15-30K', '25-50K', '20-40K', '5-10K', '20-40K', '20-30K', '10-15K', '30-50K', '6-10K', '15-30K', '15-30K', '12-20K', '15-20K', '20-35K', '50-80K', '30-40K', '15-30K', '12-18K', '11-20K', '17-30K', '18-36K', '15-30K', '15-30K', '6-10K', '18-35K', '30-45K', '20-40K', '5-8K', '12-15K', '5-10K', '6-11K', '15-30K', '20-25K', '20-35K', '16-30K', '150-200/天', '8-12K', '6-8K', '15-20K', '20-40K', '10-15K', '15-25K', '15-30K', '15-30K', '10-1

In [43]:
import json 
with open('bossZhipin_广东佛山.json',mode='w') as js:
    js.write(json.dumps(job_data))

In [44]:
with open('bossZhipin_广东佛山.json',mode='r') as boss:
    print(json.loads(boss.read()))

{'salary': ['22-35K', '20-40K', '15-30K', '13-25K', '40-70K', '25-50K', '30-60K', '20-40K', '10-15K', '25-50K', '12-24K', '20-35K', '20-40K', '20-40K', '20-40K', '15-30K', '20-30K', '35-65K', '50-80K', '30-60K', '25-50K', '13-25K', '6-10K', '12-20K', '18-28K', '20-40K', '20-35K', '12-20K', '20-35K', '50-80K', '15-30K', '15-30K', '12-20K', '25-50K', '15-25K', '20-30K', '15-30K', '17-30K', '160-210/天', '10-15K', '30-60K', '20-35K', '15-30K', '30-60K', '15-25K', '15-25K', '35-50K', '15-30K', '12-24K', '6-10K', '6-10K', '15-30K', '25-50K', '15-30K', '25-50K', '15-30K', '25-50K', '20-40K', '5-10K', '20-40K', '20-30K', '10-15K', '30-50K', '6-10K', '15-30K', '15-30K', '12-20K', '15-20K', '20-35K', '50-80K', '30-40K', '15-30K', '12-18K', '11-20K', '17-30K', '18-36K', '15-30K', '15-30K', '6-10K', '18-35K', '30-45K', '20-40K', '5-8K', '12-15K', '5-10K', '6-11K', '15-30K', '20-25K', '20-35K', '16-30K', '150-200/天', '8-12K', '6-8K', '15-20K', '20-40K', '10-15K', '15-25K', '15-30K', '15-30K', '10-1

In [7]:
from pyecharts import options as opts
from pyecharts.charts import Pie,Bar,Map,Tab,WordCloud
from pyecharts.faker import Faker

In [16]:
def pie_chart(data,title,seriers_name,width=900,height=500):
    try:
        w = str(width)+'px'
        h = str(height)+'px'
        c = (
            Pie(init_opts=opts.InitOpts(width=w,height=h))
            .add(seriers_name,data)
            .set_global_opts(title_opts=opts.TitleOpts(title=title),
                             toolbox_opts=opts.ToolboxOpts(is_show=True,orient='vertical'))
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
        )
        return c
    except ValueError:
        print("请输入正确的数据格式！比如：[('本科', 290), ('大专', 88), ('学历不限', 16), ('高中', 5), ('中专/中技', 1)]")


In [18]:
result = countNum('物联网','education')
pie_chart(result,'物联网岗位\n工资水平统计','该岗位的工资统计').render_notebook()

In [102]:
def bar_chart(ex_data,title='',seriers_name='',color='red',width=900,height=500):
    xaxis = list(dict(ex_data).keys())
    yaxis = list(dict(ex_data).values())
    w = str(width)+'px'
    h = str(height)+'px'
    c = (
        Bar(init_opts=opts.InitOpts(width=w,height=h))
        .add_xaxis(xaxis)
        .add_yaxis(seriers_name,yaxis,color=color)
  #      .set_series_opts(label_opts=opts.LabelOpts(color='blue'))
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            datazoom_opts=opts.DataZoomOpts()
        )
    )
    return c


In [110]:
def divid_salary(data):
    # 依据个人所得税税率表来将salary数据分组，否则太多太杂了。
    # 以月薪、千元为单位
    if isinstance(data,dict):
        sk = {'<3k':0,'3-12k':0,'12-25k':0,'25-35k':0,'35-55k':0,'55-80k':0,'>80k':0}
        for s in data:
            # 由于我们的产品目标是为职场小白提供择业意见，小白往往只能获得最低工资，所以我们仅选取了最低工资，方便将每个工资范围进行分组
            sl = s.split('-')
            if sl[1][-1] == 'K':
                key = int(sl[0])
                if key <=3:
                    sk['<3k'] += data[s]
                elif key > 3 and key <= 12:
                    sk['3-12k'] += data[s]
                elif key > 12 and key <= 25:
                    sk['12-25k'] += data[s]
                elif key > 25 and key <= 35:
                    sk['25-35k'] += data[s]
                elif key > 35 and key <= 55:
                    sk['35-55k'] += data[s]
                elif key > 55 and key <= 80:
                    sk['55-80k'] += data[s]
                elif key > 80:
                    sk['>80k'] +=data[s]
        result = sorted(sk.items(),key=lambda x:x[1],reverse=True)
        return result
    else:
        print('请输入字典类型的数据。')

In [29]:

# 要单独将地图可视化放到另一个页面才行。
def get_mapData(jobName):
    city_json = 'https://www.zhipin.com/wapi/zpCommon/data/city.json'
    adv = r.get(city_json,
                    headers={'User-Agent':agent}).json()['zpData']['cityList']

    # 创建一个市级行政区域与省级行政区的映射，方便后面统计各省的职位数量
    pt = {}
    for ad in adv:
        for a in ad['subLevelModelList']:
            pt[a['name']] = ad['name']

    # 先统计每个城市的职位数量，再统计省级行政区的职位数量
    pycity = count_num(get_job(jobName,100010000),'city')
    map_data = {}
    for p in pycity:
        key = pt[p[0]]
        map_data[key]= map_data.get(key,0)+p[1]  
    return map_data.items()

def map_chart(data,series_name,title,width=900,height=500):
    mapData = list(data)
    w = str(width)+'px'
    h = str(height)+'px'
    c = (
        Map(init_opts=opts.InitOpts(width=w,height=h))
        .add(series_name,mapData)
        .set_global_opts(
            title_opts=opts.TitleOpts(title=title),
            visualmap_opts=opts.VisualMapOpts(max_= mapData[0][1])
        )
    )  
    return c

map_chart(get_mapData('产品经理'),'该岗位的需求数量：','产品经理岗位需求').render_notebook()

In [37]:
salary = dict(count_num(job,'salary'))
sa_bar = bar_chart(divid_salary(salary),'产品经理薪资水平统计','该薪资水平的数量统计',color='red')
ex_bar = bar_chart(count_num(job,'experience'),color='black')
edu_pie = pie_chart(count_num(job,'education'),'产品经理的学历要求','该学历要求的数量统计')

def tab_charts(chart_list,name_list):
    tab = Tab()
    count = 1
    for cl in chart_list:
        tab.add(cl,count)
        count +=1
    return tab
tab_charts([sa_bar,ex_bar,edu_pie]).render_notebook()

In [4]:
import glob
import json

In [5]:
# fkey [大数据 人工智能 区块链 物联网 云计算 ]
# kw ['salary' 'education' 'jobname' 'experience']
def countNum(fkey,kw):
    try:
        file = glob.glob('json/'+fkey+'_*_*.json')
        edu_num = {kw:{}}
        for ff in file:
            with open(ff) as f:
                content = json.loads(f.read())
                for j in content[kw]:
                    edu_num[kw][j] = edu_num[kw].get(j,0)+1
        return sorted(edu_num[kw].items(),key= lambda x:x[1],reverse=True)
    except KeyError:
        print("请输入正确的关键词\n fkey [大数据 人工智能 区块链 物联网 云计算 ]\n 请输入正确的关键词\n fkey [大数据 人工智能 区块链 物联网 云计算 ]\n")

In [7]:
result = countNum('物联网','education')
pie_chart(result,'物联网岗位的工资水平统计','该岗位的工资统计').render_notebook()

NameError: name 'pie_chart' is not defined

In [111]:
salary_result = countNum('物联网','salary')
sa = divid_salary(dict(salary_result))
bar_chart(sa,'物联网薪资水平统计','该岗位的薪资水平').render_notebook()

In [8]:
result = countNum('物联网','jobName')
result

[('物联网产品经理', 54),
 ('物联网工程师', 47),
 ('物联网开发工程师', 37),
 ('Java', 37),
 ('产品经理', 36),
 ('销售经理', 34),
 ('物联网硬件工程师', 29),
 ('售前工程师', 27),
 ('销售专员', 27),
 ('实习生', 26),
 ('嵌入式软件工程师', 22),
 ('嵌入式', 21),
 ('城市合伙人', 21),
 ('项目经理', 20),
 ('物联网架构师', 18),
 ('外贸业务员', 18),
 ('物联网软件工程师', 17),
 ('web前端', 17),
 ('硬件工程师', 17),
 ('运维工程师', 17),
 ('平面设计师', 16),
 ('美工', 15),
 ('物联网项目经理', 14),
 ('电子工程师', 14),
 ('架构师', 13),
 ('Java开发工程师', 13),
 ('技术总监', 13),
 ('嵌入式开发工程师', 13),
 ('网络工程师', 12),
 ('测试工程师', 12),
 ('销售顾问', 12),
 ('Android', 10),
 ('PHP', 10),
 ('销售工程师', 9),
 ('技术支持工程师', 9),
 ('Java工程师', 9),
 ('销售', 9),
 ('物联网嵌入式开发工程师', 9),
 ('物联网平台开发工程师', 9),
 ('软件工程师', 9),
 ('新媒体运营', 9),
 ('电商运营', 9),
 ('硬件开发工程师', 9),
 ('C++', 8),
 ('iOS', 8),
 ('产品运营', 8),
 ('技术经理', 8),
 ('嵌入式工程师', 8),
 ('单片机', 8),
 ('文案策划', 8),
 ('物联网销售', 7),
 ('产品经理（物联网）', 7),
 ('前端开发工程师', 7),
 ('产品总监', 7),
 ('物联网平台架构师', 7),
 ('运营总监', 7),
 ('总经理助理', 7),
 ('java开发工程师', 7),
 ('平面设计', 7),
 ('服务员', 7),
 ('会计', 7),
 ('物联网实习生', 6),
 ('物联网解决方案工程师', 6

In [13]:
def wordCloud_chart(data,title,series='',width=900,height=500):
    w = str(width)+'px'
    h = str(height)+'px'
    c = (
        WordCloud(init_opts=opts.InitOpts(width=w,height=h))
        .add(series,data,word_size_range=[20,100],shape='triangle-forward')
        .set_global_opts(title_opts=opts.TitleOpts(title=title))
    )
    return c
wordCloud_chart(result,'人工智能的岗位描述词','描述词数量').render_notebook()